In [10]:
#import pandas libraries as pd.
import pandas as pd

# https://files.grouplens.org/datasets/movielens/ml-25m.zip
#files/dtaasets in the above link 
movies = pd.read_csv(r"C:\Users\sudhe\OneDrive\Desktop\movie recomendation system\ml-25m\movies.csv")

In [11]:
#dataset imported
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [27]:
#data cleaning
#Function for removing extra characters other than alphanumeric
import re

def clean_title(title):
    title = re.sub("[^a-zA-Z0-9 ]", "", title)
    return title

In [28]:

movies["clean_title"] = movies["title"].apply(clean_title)

In [29]:
#title after removing extra characters
movies

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
62418,209157,We (2018),Drama,We 2018
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul 2001
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems 2018
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing 2001


In [30]:
#text vectorization
#converting titles into vector form
#in simple terms assigning numbers to movie titles to make the matching efficient and easier
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(movies["clean_title"])

In [31]:
#similarity betwwen searched vector and title vector in the dataset given by cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies.iloc[indices].iloc[::-1]
    
    return results




In [32]:
#widgets to improve UI
import ipywidgets as widgets
from IPython.display import display

movie_input=widgets.Text(
    value="Toy Story",
    description="Movie Title:",
    disabled=False
)
movie_list=widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title=data["new"]
        if len(title)>5:
            display(search(title))
            
movie_input.observe(on_type,names='value')

display(movie_input, movie_list)

Text(value='Toy Story', description='Movie Title:')

Output()

In [35]:
#Reading the ratings csv file.
ratings = pd.read_csv(r"C:\Users\sudhe\OneDrive\Desktop\movie recomendation system\ml-25m\ratings.csv")

In [24]:
#Showing the file contents.
ratings

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [37]:
#The datatypes present in the ratings dataset.
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [38]:
#finding similar usres using ratings and provided by unique id
similar_users = ratings[(ratings["movieId"]==movie_id) & (ratings["rating"]> 4)]["userId"].unique()

In [39]:
similar_users

array([    21,    187,    208, ..., 162469, 162485, 162532], dtype=int64)

In [40]:
similar_user_recs=ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"]>4)]["movieId"]

In [42]:
similar_user_recs

3741           318
3742           527
3743           541
3744           589
3745           741
             ...  
24998517     91542
24998518     92259
24998522     98809
24998523    102125
24998524    112852
Name: movieId, Length: 577796, dtype: int64

In [43]:
similar_user_recs=similar_user_recs.value_counts()/len(similar_users)
similar_user_recs=similar_user_recs[similar_user_recs>.10]

In [44]:
similar_user_recs

movieId
89745    1.000000
58559    0.573393
59315    0.530649
79132    0.519715
2571     0.496687
           ...   
47610    0.103545
780      0.103380
88744    0.103048
1258     0.101226
1193     0.100895
Name: count, Length: 193, dtype: float64

In [45]:
all_users=ratings[(ratings["movieId"].isin(similar_user_recs.index))&(ratings["rating"]>4)]

In [46]:
all_users_recs=all_users["movieId"].value_counts()/ len(all_users["userId"].unique())

In [47]:
all_users_recs

movieId
318       0.346395
296       0.288146
2571      0.247010
356       0.238136
593       0.228665
            ...   
86332     0.010142
91630     0.009324
122900    0.008573
122926    0.008070
106072    0.005289
Name: count, Length: 193, dtype: float64

In [49]:
rec_percentages = pd.concat([similar_user_recs, all_users_recs], axis=1)
rec_percentages.columns = ["similar", "all"]

In [50]:
rec_percentages

,similar,all
movieId,,
89745,1.000000,0.040459
58559,0.573393,0.148256
59315,0.530649,0.054931
79132,0.519715,0.132987
2571,0.496687,0.247010
...,...,...
47610,0.103545,0.022770
780,0.103380,0.054723
88744,0.103048,0.010383


In [51]:
rec_percentages["score"]=rec_percentages["similar"]/rec_percentages["all"]

In [52]:
rec_percentages=rec_percentages.sort_values("score", ascending=False)

In [53]:
rec_percentages

,similar,all,score
movieId,,,
89745,1.000000,0.040459,24.716368
106072,0.103711,0.005289,19.610199
122892,0.241054,0.012367,19.491770
102125,0.216534,0.012119,17.867419
88140,0.215043,0.012052,17.843074
...,...,...,...
296,0.288933,0.288146,1.002730
593,0.222830,0.228665,0.974483
527,0.199967,0.217833,0.917984


In [54]:
rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")

,similar,all,score,movieId,title,genres,clean_title
17067,1.000000,0.040459,24.716368,89745,"Avengers, The (2012)",Action|Adventure|Sci-Fi|IMAX,Avengers The 2012
20513,0.103711,0.005289,19.610199,106072,Thor: The Dark World (2013),Action|Adventure|Fantasy|IMAX,Thor The Dark World 2013
25058,0.241054,0.012367,19.491770,122892,Avengers: Age of Ultron (2015),Action|Adventure|Sci-Fi,Avengers Age of Ultron 2015
19678,0.216534,0.012119,17.867419,102125,Iron Man 3 (2013),Action|Sci-Fi|Thriller|IMAX,Iron Man 3 2013
16725,0.215043,0.012052,17.843074,88140,Captain America: The First Avenger (2011),Action|Adventure|Sci-Fi|Thriller|War,Captain America The First Avenger 2011
16312,0.175447,0.010142,17.299824,86332,Thor (2011),Action|Adventure|Drama|Fantasy|IMAX,Thor 2011
21348,0.287608,0.016737,17.183667,110102,Captain America: The Winter Soldier (2014),Action|Adventure|Sci-Fi|IMAX,Captain America The Winter Soldier 2014
25071,0.214049,0.012856,16.649399,122920,Captain America: Civil War (2016),Action|Sci-Fi|Thriller,Captain America Civil War 2016
25061,0.136017,0.008573,15.865628,122900,Ant-Man (2015),Action|Adventure|Sci-Fi,AntMan 2015
14628,0.242876,0.015517,15.651921,77561,Iron Man 2 (2010),Action|Adventure|Sci-Fi|Thriller|IMAX,Iron Man 2 2010


In [55]:
#Function to find similar movies applied on the input.
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"]==movie_id) & (ratings["rating"]> 4)]["userId"].unique()
    similar_user_recs=ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"]>4)]["movieId"]
    
    similar_user_recs=similar_user_recs.value_counts()/len(similar_users)
    similar_user_recs=similar_user_recs[similar_user_recs>.10]
    
    all_users=ratings[(ratings["movieId"].isin(similar_user_recs.index))&(ratings["rating"]>4)]
    all_users_recs=all_users["movieId"].value_counts()/ len(all_users["userId"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_users_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    
    rec_percentages["score"]=rec_percentages["similar"]/rec_percentages["all"]
    
    rec_percentages=rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")

In [58]:
movie_name_input=widgets.Text(
    value="Toy Story",
    description="Movie Title:",
    disabled=False
)

recommendation_list=widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title=data["new"]
        if len(title) >5:
            results=search(title)
            movie_id=results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names="value")

display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()